In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [34]:
df_test_original = pd.DataFrame(pd.read_excel('personalroadriskassessv2test.xlsx'))

df_test = df_test_original.drop(['SGFSSJ','WFSJ'],1)
part_df_test = df_test[[
    'ZJCX', 'LJJF', 'xb', 'jl',  'marrige', 'child', 'hnum',
    'age', 'XZQH','SSRS','SWRS7','sghpzl' , 'SFSG'
]]
# 选择参数列
X_select_elements = ['LJJF','ZJCX', 'age', 'xb', 'jl', 'child', 'marrige','XZQH','sghpzl','SSRS']
# 测试数据
test_X = np.array(part_df_test[X_select_elements])
# 均值化
minMax = MinMaxScaler()
test_X = minMax.fit_transform(test_X)

xs1 = test_X.shape[1]
# ys1 = train_Y_oh.shape[1]
ys1 = 2

In [35]:
print('xs1',xs1)
print('ys1',ys1)
tf.reset_default_graph()

x = tf.placeholder(tf.float32,shape = [None,xs1],name = 'input')
y = tf.placeholder(tf.float32,shape = [None,ys1],name = 'label')
weights1 = tf.Variable(tf.random_normal([xs1,xs1]),name = 'weights1')
bias1 = tf.Variable(tf.zeros([xs1]),name = 'bias1')
a = tf.nn.relu(tf.matmul(x,weights1) + bias1)
weights2 = tf.Variable(tf.random_normal([xs1,ys1]),name = 'weights2')
bias2 = tf.Variable(tf.zeros([ys1]),name = 'bias2')
z = tf.matmul(a,weights2) + bias2
y_pred = tf.nn.softmax(z)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=z))
# cost = tf.log(tf.clip_by_value(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=z)),1e-8,1.0))

correct_pred = tf.equal(tf.argmax(y,1),tf.argmax(y_pred,1))
acc_op = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
train_op = tf.train.AdamOptimizer(0.001).minimize(cost)
# loss = tf.log(tf.clip_by_value(y,1e-8,1.0))
# loss = tf.log(tf.clip_by_value(tf.sigmoid(self.scores),1e-8,1.0)

xs1 10
ys1 2


In [31]:
ckpt_dir = './ckpt_dir'

saver = tf.train.Saver()

with tf.Session() as sess:
#     ckpt = tf.train.latest_checkpoint(ckpt_dir)
    print('ckpt',ckpt)
    if ckpt:
        print('Restoring from checkpoint: %s' % ckpt)
#         saver.restore(sess, ckpt)
        saver.restore(sess, ckpt_dir + '/logistic.ckpt')
    
#     评估模型  
    print(y_pred)
    possibility = sess.run(y_pred, feed_dict={x: test_X})
    
    possibility_array = np.array(possibility,dtype=float)
    
#     print('possibility',possibility)
#     print('possibility.shape',possibility.shape)
#     print('possibility.shape',type(possibility.shape))
    
    predictions = np.argmax(possibility, 1)
    
#     print('possibility[0]',possibility[:,0])
#     print('possibility[0]',(possibility[:,0].shape))

    #保存结果  
    submission = pd.DataFrame({  
        "id": df_test_original["id"],  
        "SFSG": predictions,
        "possibility_0": possibility[:,0],
        "possibility_1": possibility[:,1]
    })  
    submission.to_csv("SG-submission_re.csv", index=False)

ckpt ./ckpt_dir/logistic.ckpt
Restoring from checkpoint: ./ckpt_dir/logistic.ckpt
INFO:tensorflow:Restoring parameters from ./ckpt_dir/logistic.ckpt
Tensor("Softmax:0", shape=(?, 2), dtype=float32)
